# Deploying Fraud Detection Model with KServe

In this notebook, we'll deploy our trained fraud detection model using KServe InferenceService.

In [1]:
import os
import yaml
import json
from kubernetes import client, config
from kserve import KServeClient
import time

## Build Docker Images for Custom Components

In [ ]:
# Build the transformer image
!docker build -f Dockerfile.transformer -t fraud-detection-transformer:latest .

# Build the explainer image
!docker build -f Dockerfile.explainer -t fraud-detection-explainer:latest .

## Load the Model and Deploy with KServe

Now we'll deploy the model with KServe using the InferenceService.

In [ ]:
# Load Kubernetes configuration
config.load_kube_config()

# Create KServe client
kserve_client = KServeClient()

# Load the InferenceService YAML
with open('kserve/inferenceservice.yaml', 'r') as f:
    inferenceservice = yaml.safe_load(f)

# Deploy the InferenceService
kserve_client.create(inferenceservice)

print(f"InferenceService {inferenceservice['metadata']['name']} deployed in namespace {inferenceservice['metadata']['namespace']}")

In [ ]:
# Wait for the service to be ready
service_name = inferenceservice['metadata']['name']
namespace = inferenceservice['metadata']['namespace']

def wait_for_service_ready(name, namespace, timeout_minutes=5):
    start_time = time.time()
    timeout = timeout_minutes * 60
    
    while time.time() - start_time < timeout:
        try:
            response = kserve_client.get(name, namespace=namespace)
            conditions = response.get('status', {}).get('conditions', [])
            
            # Check if the service is ready
            for condition in conditions:
                if condition.get('type') == 'Ready' and condition.get('status') == 'True':
                    print(f"Service {name} is ready!")
                    return True
            
            print(f"Waiting for service {name} to be ready...")
            time.sleep(10)
        except Exception as e:
            print(f"Error checking service status: {e}")
            time.sleep(10)
    
    print(f"Timeout waiting for service {name} to be ready")
    return False

wait_for_service_ready(service_name, namespace)

## Deploy Advanced KServe Features

In [ ]:
# Deploy with Transformer
with open('kserve/transformer.yaml', 'r') as f:
    transformer_service = yaml.safe_load(f)

# Update the existing service to add transformer
kserve_client.replace(transformer_service)

print(f"Added transformer to {service_name}")
wait_for_service_ready(service_name, namespace)

In [ ]:
# Deploy with Explainer
with open('kserve/explainer.yaml', 'r') as f:
    explainer_service = yaml.safe_load(f)

# Update the existing service to add explainer
kserve_client.replace(explainer_service)

print(f"Added explainer to {service_name}")
wait_for_service_ready(service_name, namespace)

## Get Inference Service Details

In [ ]:
# Get the URLs for the inference service
service = kserve_client.get(service_name, namespace=namespace)

# Extract service URLs
predictor_url = None
transformer_url = None
explainer_url = None

if 'status' in service and 'url' in service['status']:
    base_url = service['status']['url']
    
    # Predictor URL
    predictor_url = f"{base_url}/v1/models/{service_name}:predict"
    
    # Transformer URL (if exists)
    if 'transformer' in service['spec']:
        transformer_url = f"{base_url}/v1/models/{service_name}:predict"
    
    # Explainer URL (if exists)
    if 'explainer' in service['spec']:
        explainer_url = f"{base_url}/v1/models/{service_name}:explain"

print(f"Predictor URL: {predictor_url}")
print(f"Transformer URL: {transformer_url}")
print(f"Explainer URL: {explainer_url}")